# Project: Human Resources Dataset Analysis

## Week 2: (1) Exploratory Data Analysis (EDA) And Determine Data Analysis Questions

In [53]:
# import Libraries
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc

pd.options.display.max_rows = None

pd.options.display.max_columns = None
sns.set()


In [55]:
import pyodbc
import pandas as pd

def read_sql_query(query):
    
    server = 'DESKTOP-0CQ5N9B' 
    database = 'HR_system' 
    
    # SQL Authentication
    connection_string = (
        f"Driver={{ODBC Driver 17 for SQL Server}};" 
        f"Server={server};" 
        f"Database={database};"
        f"Trusted_Connection=yes;" 
    )

    # Creating connection
    try:
        connection = pyodbc.connect(connection_string) 
        print("Connection successful!")

        # Use cursor to execute query
        cursor = connection.cursor()
        cursor.execute(query)
        
        # Get results
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description] 
        
        # Creating DataFrame
        df = pd.DataFrame.from_records(rows, columns=columns)

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None  
    
    finally:
        # close connection
        if 'connection' in locals():  
            connection.close()


### Employees:

In [73]:
pth= "../00-Dataset_Data_Model/"
# employees = pd.read_csv(f"{pth}01-Employees.csv")
# educationlevels = pd.read_csv(f"{pth}03-EducationLevels.csv")


## Load Dataset of Employees with EducationLevels as categories
query = """ 
SELECT 
    E.*,
    EL.EducationLevel AS EducationLevel
FROM 
    Employees E
LEFT JOIN 
    EducationLevels EL ON E.Education = EL.EducationLevelID;
"""
employees = read_sql_query(query)

employees.head()

Connection successful!


,EmployeeID,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome,State,Ethnicity,Education,EducationField,JobRole,MaritalStatus,Salary,StockOptionLevel,OverTime,HireDate,Attrition,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager,EducationLevel
0,001A-8F88,Christy,Jumel,Male,22,Some Travel,Technology,40,CA,White,4,Information Systems,Software Engineer,Married,27763.00,0,No,2021-09-05,No,1,0,1,0,Masters
1,005C-E0FB,Fin,O'Halleghane,Non-Binary,24,Frequent Traveller,Sales,17,CA,White,4,Marketing,Sales Executive,Married,56155.00,1,No,2017-08-26,No,5,2,2,0,Masters
2,00A3-2445,Wyatt,Ziehm,Male,30,Some Travel,Technology,6,CA,Black or African American,2,Computer Science,Machine Learning Engineer,Married,126238.00,0,No,2012-03-08,No,10,3,6,6,High School
3,00B0-F199,Trueman,Jirasek,Male,23,Some Travel,Sales,35,CA,White,1,Marketing,Sales Executive,Married,97824.00,1,Yes,2020-03-16,Yes,1,0,1,0,No Formal Qualifications
4,00D4-DD53,Joyce,Goor,Female,30,Frequent Traveller,Technology,44,CA,Black or African American,1,Computer Science,Software Engineer,Single,68508.00,0,Yes,2012-01-28,Yes,5,4,4,4,No Formal Qualifications


In [9]:
# Display employees data information
employees.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EmployeeID               1470 non-null   object
 1   FirstName                1470 non-null   object
 2   LastName                 1470 non-null   object
 3   Gender                   1470 non-null   object
 4   Age                      1470 non-null   int64 
 5   BusinessTravel           1470 non-null   object
 6   Department               1470 non-null   object
 7   DistanceFromHome (KM)    1470 non-null   int64 
 8   State                    1470 non-null   object
 9   Ethnicity                1470 non-null   object
 10  Education                1470 non-null   int64 
 11  EducationField           1470 non-null   object
 12  JobRole                  1470 non-null   object
 13  MaritalStatus            1470 non-null   object
 14  Salary                   1470 non-null  

In [76]:
# Summary statistics
employees.describe()


,Age,DistanceFromHome,Education,StockOptionLevel,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,28.989796,22.502721,2.912925,0.793878,4.562585,2.293197,3.440816,2.239456
std,7.993055,12.811124,1.024165,0.852077,3.288048,2.539093,2.945194,2.505774
min,18.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,12.000000,2.000000,0.000000,2.000000,0.000000,1.000000,0.000000
50%,26.000000,22.000000,3.000000,1.000000,4.000000,1.000000,3.000000,1.000000
75%,34.000000,33.000000,4.000000,1.000000,7.000000,4.000000,6.000000,4.000000
max,51.000000,45.000000,5.000000,3.000000,10.000000,10.000000,10.000000,10.000000


____________________________

____________________________

__________________________

__________________________

### PerformanceRating:

In [75]:
# Reading Dataset of PerformanceRating.
#performancerating = pd.read_csv(f"{pth}02-PerformanceRating.csv")
#satisfactionlevels = pd.read_csv(f"{pth}04-SatisfactionLevels.csv")
#ratinglevels = pd.read_csv(f"{pth}05-RatingLevels.csv")


## Load Dataset of PerformanceRating with SatisfactionLevels and RatingLevels as categories
query = """ 
SELECT 
    PR.*,
    SL1.SatisfactionLevel AS EnvironmentSatisfaction,
    SL2.SatisfactionLevel AS JobSatisfaction,
    SL3.SatisfactionLevel AS RelationshipSatisfaction,
    SL4.SatisfactionLevel AS WorkLifeBalance,
    RL1.RatingLevel AS SelfRating,
    RL2.RatingLevel AS ManagerRating
FROM 
    PerformanceRating PR
LEFT JOIN 
    SatisfactionLevels SL1 ON PR.EnvironmentSatisfaction = SL1.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL2 ON PR.JobSatisfaction = SL2.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL3 ON PR.RelationshipSatisfaction = SL3.SatisfactionID
LEFT JOIN 
    SatisfactionLevels SL4 ON PR.WorkLifeBalance = SL4.SatisfactionID
LEFT JOIN 
    RatingLevels RL1 ON PR.SelfRating = RL1.RatingID
LEFT JOIN 
    RatingLevels RL2 ON PR.ManagerRating = RL2.RatingID;
"""
performancerating = read_sql_query(query)

performancerating.head()

Connection successful!


,PerformanceID,EmployeeID,ReviewDate,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,WorkLifeBalance,SelfRating,ManagerRating,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance,SelfRating,ManagerRating
0,PR01,79F7-78EC,2013-01-02,5,4,5,1,0,4,4,4,Very Satisfied,Satisfied,Very Satisfied,Satisfied,Exceeds Expectation,Exceeds Expectation
1,PR02,B61E-0F26,2013-01-03,5,4,4,1,3,4,4,3,Very Satisfied,Satisfied,Satisfied,Satisfied,Exceeds Expectation,Meets Expectation
2,PR03,F5E3-48BB,2013-01-03,3,4,5,3,2,3,5,4,Neutral,Satisfied,Very Satisfied,Neutral,Above and Beyond,Exceeds Expectation
3,PR04,0678-748A,2013-01-04,5,3,2,2,0,2,3,2,Very Satisfied,Neutral,Dissatisfied,Dissatisfied,Meets Expectation,Needs Improvement
4,PR05,541F-3E19,2013-01-04,5,2,3,1,0,4,4,3,Very Satisfied,Dissatisfied,Neutral,Satisfied,Exceeds Expectation,Meets Expectation


In [77]:
# Summary statistics
employees.describe()


,Age,DistanceFromHome,Education,StockOptionLevel,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,28.989796,22.502721,2.912925,0.793878,4.562585,2.293197,3.440816,2.239456
std,7.993055,12.811124,1.024165,0.852077,3.288048,2.539093,2.945194,2.505774
min,18.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,12.000000,2.000000,0.000000,2.000000,0.000000,1.000000,0.000000
50%,26.000000,22.000000,3.000000,1.000000,4.000000,1.000000,3.000000,1.000000
75%,34.000000,33.000000,4.000000,1.000000,7.000000,4.000000,6.000000,4.000000
max,51.000000,45.000000,5.000000,3.000000,10.000000,10.000000,10.000000,10.000000


_____________________________________

## (2) Data Cleaning:
After building the data model, we proceeded with data cleaning and preprocessing. Here’s a summary of the key observations:

- The data had no missing values or unusual entries across all tables. Each field, such as age, salary, and years of experience, showed values within expected ranges.
- Data entries for categorical variables like gender, marital status, and job roles were consistent without any spelling or formatting issues.
- Date fields, such as the employee `HireDate` and `ReviewDate` in the performance review table, were in the correct format and adhered to the expected chronological order.
- Numeric fields, including `Salary`, `YearsAtCompany`, and `DistanceFromHome`, were confirmed to contain only valid numbers without any outliers or inconsistent values.

## Conclusion:
The dataset was thoroughly examined and found to be clean, consistent, and aligned with the designed data model. There were no missing values, illogical entries, or repeated values, ensuring data integrity across all tables. 


- **Python (pandas, Matplotlib)**: For detailed data Cleaning and visual inspection.

# END